In [695]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import TweedieRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from sklearn.metrics import r2_score
import pickle

df= pd.read_csv('Datos/Limpios/df_valoracion.csv')
lista_variables= ['growth_stage', 'startup', 'b2b_b2c', 'Nombre_sabi']
df = df.drop(lista_variables, axis=1)
X_train, X_test, y_train, y_test = train_test_split(df.drop('valuation_2022', axis=1), 
                                                    df['valuation_2022'], 
                                                    test_size=0.2, 
                                                    random_state=0)

In [696]:
y_train_log = np.log(y_train)


models = {'Linear Regression': LinearRegression(), 
          'Decision Tree': DecisionTreeRegressor(random_state=42), 
          'Random Forest': RandomForestRegressor(random_state=42)}

for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train_log.ravel(), cv=5, scoring='r2')
    print(f"{name} R2 score: {scores.mean():.2f} (+/- {scores.std():.2f})")
    print(f"{name} MAE score: {-cross_val_score(model, X_train, y_train_log.ravel(), cv=5, scoring='neg_mean_absolute_error').mean():.2f} (+/- {-cross_val_score(model, X_train, y_train_log.ravel(), cv=5, scoring='neg_mean_absolute_error').std():.2f})")


results = []

for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train_log.ravel(), cv=5, scoring='r2')
    mae_scores = -cross_val_score(model, X_train, y_train_log.ravel(), cv=5, scoring='neg_mean_absolute_error')
    result = {'model': name,
              'R2 score': scores.mean().round(2),
              'MAE score': -cross_val_score(model, X_train, y_train_log.ravel(), cv=5, scoring='neg_mean_absolute_error').mean().round(2)
              }
    results.append(result)    

Linear Regression R2 score: -206.78 (+/- 413.59)
Linear Regression MAE score: 2.84 (+/- -3.71)
Decision Tree R2 score: 0.66 (+/- 0.22)
Decision Tree MAE score: 0.41 (+/- -0.12)
Random Forest R2 score: 0.78 (+/- 0.12)
Random Forest MAE score: 0.41 (+/- -0.09)


In [697]:
results

[{'model': 'Linear Regression', 'R2 score': -206.78, 'MAE score': 2.84},
 {'model': 'Decision Tree', 'R2 score': 0.66, 'MAE score': 0.41},
 {'model': 'Random Forest', 'R2 score': 0.78, 'MAE score': 0.41}]

In [698]:
linear_regression_mae = results[0]['MAE score']
linear_regression_r2 = results[0]['R2 score']
decision_tree_mae = results[1]['MAE score']
decision_tree_r2 = results[1]['R2 score']
random_forest_mae = results[2]['MAE score']
random_forest_r2 = results[2]['R2 score']

In [699]:
# stacking 
level0 = [
    ('knn', KNeighborsRegressor()),
    ('cart', DecisionTreeRegressor()),
    ('svm', SVR()),
    ('lr', LinearRegression())
]

# Define the level 1 model with a transformed target regressor
level1 = TweedieRegressor(max_iter=10000, alpha=0.5, link='log', power=0)

# Define the stacked model
stacked_model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)

# Fit the model using cross-validation
scores = -cross_val_score(stacked_model, X_train, y_train_log, cv=5, scoring='neg_mean_absolute_error')
scores2 = cross_val_score(stacked_model, X_train, y_train_log, cv=5, scoring='r2')
avg_mae2 = scores.mean()
avg_r22 = scores2.mean()

# Fit the model on all the training data
stacked_model.fit(X_train, y_train_log)

# Make predictions on the test data and transform the predictions to the original scale
y_pred = np.exp(stacked_model.predict(X_test))

# Evaluate the model on the test data
mae2 = mean_absolute_error(y_test, y_pred)
r22 = r2_score(y_test, y_pred)

# Print the results
print(f"Cross-validation MAE: {avg_mae2:.2f}")
print(f"Cross-validation R^2: {avg_r22:.2f}")
print(f"MAE: {mae2:.2f}")
print(f"R^2: {r22:.2f}")

Cross-validation MAE: 0.93
Cross-validation R^2: 0.36
MAE: 12.31
R^2: 0.16


In [700]:
#Bagging Regressor
br = BaggingRegressor(estimator=RandomForestRegressor(), n_estimators=100, random_state=0)
scores = cross_val_score(br, X_train, y_train_log, cv=5, scoring='neg_mean_absolute_error')
scores2 = cross_val_score(br, X_train, y_train_log, cv=5, scoring='r2')
avg_mae7 = -scores.mean()
avg_r27 = scores2.mean()

# ajustar el modelo a todos los datos de entrenamiento
model.fit(X_train, y_train_log)
# hacer predicciones en los datos de prueba y desescalar las predicciones
y_pred = np.exp(model.predict(X_test))

# evaluar el modelo en los datos de prueba
mae7 = mean_absolute_error(y_test, y_pred)
r27 = r2_score(y_test, y_pred)

# imprimir los resultados
print(f"Cross-validation MAE: {avg_mae7}")
print(f'R2: {avg_r27}')
print(f"MAE: {mae7}")
print(f"R^2: {r27}")

Cross-validation MAE: 0.5262145576822407
R2: 0.7599954392353386
MAE: 11.262176566366245
R^2: 0.1264634717756472


In [701]:
ab = AdaBoostRegressor(random_state=0)
scores = cross_val_score(ab, X_train, y_train_log, cv=5, scoring='neg_mean_absolute_error')
scores2 = cross_val_score(ab, X_train, y_train_log, cv=5, scoring='r2')
avg_mae9 = -scores.mean()
avg_r29 = scores2.mean()

# ajustar el modelo a todos los datos de entrenamiento
model.fit(X_train, y_train_log)
# hacer predicciones en los datos de prueba y desescalar las predicciones
y_pred = np.exp(model.predict(X_test))

# evaluar el modelo en los datos de prueba
mae9 = mean_absolute_error(y_test, y_pred)
r29 = r2_score(y_test, y_pred)

# imprimir los resultados
print(f"Cross-validation MAE: {avg_mae9}")
print(f'R2: {avg_r29}')
print(f"MAE: {mae9}")
print(f"R^2: {r29}")

Cross-validation MAE: 0.4728705496872404
R2: 0.7421827682914215
MAE: 11.262176566366245
R^2: 0.1264634717756472


In [702]:
# gradient boosting
gb = GradientBoostingRegressor(random_state=0)
scores = cross_val_score(gb, X_train, y_train_log, cv=5, scoring='neg_mean_absolute_error')
scores2= cross_val_score(gb, X_train, y_train_log, cv=5, scoring='r2')
avg_mae10 = -scores.mean()
avg_r210 = scores2.mean()

# ajustar el modelo a todos los datos de entrenamiento
model.fit(X_train, y_train_log)
# hacer predicciones en los datos de prueba y desescalar las predicciones
y_pred = np.exp(model.predict(X_test))

# evaluar el modelo en los datos de prueba
mae10 = mean_absolute_error(y_test, y_pred)
r210 = r2_score(y_test, y_pred)

# imprimir los resultados
print(f"Cross-validation MAE: {avg_mae10}")
print(f'R2: {avg_r210}')
print(f"MAE: {mae10}")
print(f"R^2: {r210}")

Cross-validation MAE: 0.32703884612077216
R2: 0.7785748022140995
MAE: 11.262176566366245
R^2: 0.1264634717756472


In [703]:
# xgboost
xgb = XGBRegressor(random_state=0)
scores = cross_val_score(xgb, X_train, y_train_log, cv=5, scoring='neg_mean_absolute_error')
scores2 = cross_val_score(xgb, X_train, y_train_log, cv=5, scoring='r2')
avg_mae11 = -scores.mean()
avg_r211 = scores2.mean()

# ajustar el modelo a todos los datos de entrenamiento
model.fit(X_train, y_train_log)
# hacer predicciones en los datos de prueba y desescalar las predicciones
y_pred = np.exp(model.predict(X_test))

# evaluar el modelo en los datos de prueba
mae11 = mean_absolute_error(y_test, y_pred)
r211 = r2_score(y_test, y_pred)

# imprimir los resultados
print(f"Cross-validation MAE: {avg_mae11}")
print(f'R2: {avg_r211}')
print(f"MAE: {mae11}")
print(f"R^2: {r211}")

Cross-validation MAE: 0.31750211769910175
R2: 0.7806804171492269
MAE: 11.262176566366245
R^2: 0.1264634717756472


In [704]:
# catboost
cb = CatBoostRegressor(random_state=0)
scores = cross_val_score(cb, X_train, y_train_log, cv=5, scoring='neg_mean_absolute_error')
scores2 = cross_val_score(cb, X_train, y_train_log, cv=5, scoring='r2')
avg_mae12 = -scores.mean()
avg_r212 = scores2.mean()

# ajustar el modelo a todos los datos de entrenamiento
model.fit(X_train, y_train_log)
# hacer predicciones en los datos de prueba y desescalar las predicciones
y_pred = np.exp(model.predict(X_test))

# evaluar el modelo en los datos de prueba
mae12 = mean_absolute_error(y_test, y_pred)
r212 = r2_score(y_test, y_pred)

# imprimir los resultados
print(f"Cross-validation MAE: {avg_mae12}")
print(f'R2: {avg_r212}')
print(f"MAE: {mae12}")
print(f"R^2: {r212}")

Learning rate set to 0.027076
0:	learn: 1.6269403	total: 1.9ms	remaining: 1.9s
1:	learn: 1.6037787	total: 3.34ms	remaining: 1.67s
2:	learn: 1.5795280	total: 4.78ms	remaining: 1.59s
3:	learn: 1.5568459	total: 6.25ms	remaining: 1.55s
4:	learn: 1.5332614	total: 7.67ms	remaining: 1.53s
5:	learn: 1.5129629	total: 9.08ms	remaining: 1.5s
6:	learn: 1.4917007	total: 10.5ms	remaining: 1.48s
7:	learn: 1.4690150	total: 11.9ms	remaining: 1.48s
8:	learn: 1.4497593	total: 13.3ms	remaining: 1.47s
9:	learn: 1.4330707	total: 14.7ms	remaining: 1.46s
10:	learn: 1.4149075	total: 16.2ms	remaining: 1.45s
11:	learn: 1.3922292	total: 17.6ms	remaining: 1.45s
12:	learn: 1.3700429	total: 19ms	remaining: 1.45s
13:	learn: 1.3514819	total: 20.5ms	remaining: 1.44s
14:	learn: 1.3305616	total: 21.9ms	remaining: 1.44s
15:	learn: 1.3155433	total: 23.3ms	remaining: 1.43s
16:	learn: 1.2979775	total: 24.8ms	remaining: 1.43s
17:	learn: 1.2789410	total: 26.2ms	remaining: 1.43s
18:	learn: 1.2619085	total: 27.6ms	remaining: 1.4

In [705]:
import pickle

In [707]:
# se guardan los modelos que mejor resultado han dado
# se crea carpeta de modelos si no existe
if not os.path.exists('modelos'):
    os.makedirs('modelos')

# dentro de esa carpeta se crea otra carpeta para los modelos de regresion
if not os.path.exists('modelos/regresion'):
    os.makedirs('modelos/regresion')

# se guarda el modelo de bagging  en la carpeta de modelos
pickle.dump(br, open('modelos/regresion/br_model.pkl', 'wb'))

# se guarda el modelo de adaboost  en la carpeta de modelos
pickle.dump(ab, open('modelos/regresion/ad_model.pkl', 'wb'))

# se guarda el modelo de gradientboost  en la carpeta de modelos
pickle.dump(gb, open('modelos/regresion/gb_model.pkl', 'wb'))

# se guarda el modelo de xgboost  en la carpeta de modelos
pickle.dump(xgb, open('modelos/regresion/gb_model.pkl', 'wb'))

# se guarda el modelo de catboost  en la carpeta de modelos
pickle.dump(cb, open('modelos/regresion/cb_model.pkl', 'wb'))

**TABLA DE RESULTADOS**

In [708]:
models = pd.DataFrame([
    {'Model': 'Linear Regression', 'MAE': linear_regression_mae, 'R2 score': linear_regression_r2},
    {'Model': 'Decision Tree Regressor', 'MAE': decision_tree_mae, 'R2 score': decision_tree_r2},
    {'Model': 'Random Forest Regressor', 'MAE': random_forest_mae, 'R2 score': random_forest_r2},
    {'Model': 'Stacking Regressor', 'MAE': avg_mae2, 'R2 score': avg_r22},
    {'Model': 'Bagging Regressor', 'MAE': avg_mae7, 'R2 score': avg_r27},
    {'Model': 'AdaBoost Regressor', 'MAE': avg_mae9, 'R2 score': avg_r29},
    {'Model': 'GradientBoost Regressor', 'MAE': avg_mae10, 'R2 score': avg_r210},
    {'Model': 'XGBoost', 'MAE': avg_mae11, 'R2 score': avg_r211},
    {'Model': 'CatBoost', 'MAE': avg_mae12, 'R2 score': avg_r212}
    # {'Model': 'AutoML', 'MAE': avg_mae17, 'Score': avg_r217}
])

models = models.sort_values(by='R2 score', ascending=False)
models = models.reindex(columns=['Model', 'R2 score', 'MAE'])
models

,Model,R2 score,MAE
7,XGBoost,0.780680,0.317502
2,Random Forest Regressor,0.780000,0.410000
8,CatBoost,0.779167,0.343884
6,GradientBoost Regressor,0.778575,0.327039
4,Bagging Regressor,0.759995,0.526215
5,AdaBoost Regressor,0.742183,0.472871
1,Decision Tree Regressor,0.660000,0.410000
3,Stacking Regressor,0.357886,0.934560
0,Linear Regression,-206.780000,2.840000
